**Лабораторный практикум по курсу «Распознавание диктора», Университет ИТМО, 2025**		

**Лабораторная работа №6. Обучение блока построения дикторских моделей на основе трансформерных претрейнов**

**Цель работы:** изучение процедуры обучения блока построения дикторских моделей на основе трансформерных претрейнов.

**Краткое описание:** в рамках настоящей лабораторной работы предлагается изучить и реализовать схему обучения нейросетевого блока построения дикторских моделей на основе Whisper tiny трансформерного претрейна. Процедуры обучения и тестирования предлагается выполнить по отношению к задаче идентификации на закрытом множестве и верификации на открытом множестве, соответственно. Тестирование полученной системы предполагает использование доли правильных ответов (accuracy) в рамках идентификации диктора на закрытом множестве, а также равновероятной ошибки (EER, equal error rate) в рамках верификации диктора на открытом множестве в качестве целевых метрик оценки качества.

**Данные:** в качестве данных для выполнения лабораторной работы предлагается использовать базу [VoxCeleb1](http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html).

**Содержание лабораторной работы**

1. Подготовка данных для обучения и тестирования блока построения дикторских моделей.							

2. Обучение параметров блока построения дикторских моделей, основанного на использовании «размороженного» Whisper tiny трансформерного претрейна.

3. Проведение дополнительных экспериментов по использованию трансформерных архитектур в задаче распознавания диктора.

In [1]:
# IPython extension to reload modules before executing user code
%load_ext autoreload
%autoreload 2

# Import of modules
import os
import sys
import itertools
from collections import OrderedDict

sys.path.append(os.path.realpath('..'))

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from common import download_dataset, concatenate, extract_dataset, part_extract, download_protocol, split_musan
from common import test_dataset_loader as test_dataset_loader_verification
from common import extract_features, compute_scores
from common import get_eer
from exercises_blank import train_dataset_loader, test_dataset_loader, train_network, test_network, tar_imp_hists
from exercises_blank import WhisperEncoder, TDNNSimple, StatPoolLayer, MaxoutSegmentLevel, SSLDownstream, MainModel
from whisper_fbanks import ExtractWhisperFbanks80
from LossFunction import AAMSoftmaxLoss
from Optimizer import SGDOptimizer, AdamOptimizer
from Scheduler import OneCycleLRScheduler
from load_save_pth import saveParameters, loadParameters

**1. Подготовка данных для обучения и тестирования детектора речевой активности**

В ходе выполнения лабораторной работы необходимы данные для проведения процедуры обучения и процедуры тестирования нейросетевого блока генерации дикторских моделей. Возьмём в качестве этих данных звукозаписи, сохраненные в формат *wav*, из корпуса [VoxCeleb1 dev set и VoxCeleb1 test set](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html). Данные корпуса содержат 148,642 и 4,874 звукозаписи (частота дискретизации равна 16кГц) с общим количеством дикторов женского и мужского пола равным 1,251, разговаривающих преимущественно на английском языке. В рамках настоящей лабораторной работы данные из корпуса VoxCeleb1 dev set  будут использоваться при проведения процедур обучения и валидации блока построения дикторских моделей в задаче идентификации на закрытом множестве. Данные из корпуса VoxCeleb1 test set будут использоваться для проведения верификационного теста на открытом множестве из 40 дикторов, которые отсутствовали в корпусе VoxCeleb1 dev set.

В рамках настоящего пункта требуется выполнить следующее:

1. Загрузить и распаковать звуковые wav-файлы из корпуса VoxCeleb1 dev set и VoxCeleb1 test set.

2. Загрузить идентификационный протокол, позволяющий разделить данные из корпуса VoxCeleb1 dev set на обучающее и валидацилонное множества.

3. Загрузить верификационный протокол, связанный с корпусом VoxCeleb1 test set.

4. Загрузить и распаковать данные из корпусов [SLR17 (MUSAN)](https://openslr.org/17/) and [SLR28 (RIR noises)](https://openslr.org/28/), с использованием которых можно расширить обучающее множество данных через запуск процедур аугментации.

![Рисунок 1](https://analyticsindiamag.com/wp-content/uploads/2020/12/image.png "VoxCeleb. Крупномасштабная аудиовизуальная база данных человеческой речи.")

In [2]:
# Download VoxCeleb1 (test set)
with open('../data/lists/datasets.txt', 'r') as f:
    lines = f.readlines()

download_dataset(lines, user='voxceleb1902', password='nx0bl2v2', save_path='../data')

Checksum successful vox1_test_wav.zip.
Checksum successful vox1_dev_wav_partaa.
Checksum successful vox1_dev_wav_partab.
Checksum successful vox1_dev_wav_partac.
Checksum successful vox1_dev_wav_partad.


In [3]:
# Concatenate archives for VoxCeleb1 dev set
with open('../data/lists/concat_arch.txt', 'r') as f:
    lines = f.readlines()
    
concatenate(lines, save_path='../data')

Checksum successful vox1_dev_wav.zip.


In [4]:
# Extract VoxCeleb1 dev set
extract_dataset(save_path='../data/voxceleb1_dev', fname='../data/vox1_dev_wav.zip')

Extracting of ../data/vox1_dev_wav.zip is successful.


In [5]:
# Download VoxCeleb1 identification protocol and VoxCeleb1-O cleaned protocol
with open('../data/lists/protocols.txt', 'r') as f:
    lines = f.readlines()
    
download_protocol(lines, save_path='../data/voxceleb1_test')

File veri_test2.txt is downloaded.
File iden_split.txt is downloaded.


In [6]:
# Download SLR17 (MUSAN) and SLR28 (RIR noises) datasets for train data augmentation
with open('../data/lists/augment_datasets.txt', 'r') as f:
    lines = f.readlines()
    
download_dataset(lines, user=None, password=None, save_path='../data')

Checksum successful musan.tar.gz.
Checksum successful rirs_noises.zip.


In [7]:
# Extract SLR17 (MUSAN)
extract_dataset(save_path='../data', fname='../data/musan.tar.gz')

# Extract SLR28 (RIR noises)
part_extract(save_path='../data', fname='../data/rirs_noises.zip', target=['RIRS_NOISES/simulated_rirs/mediumroom', 'RIRS_NOISES/simulated_rirs/smallroom'])

Extracting of ../data/musan.tar.gz is successful.
Extracting ../data/rirs_noises.zip


In [8]:
# Split MUSAN (SLR17) dataset for faster random access
split_musan(save_path='../data')

0 ../data/musan/noise/free-sound/noise-free-sound-0536.wav
1 ../data/musan/noise/free-sound/noise-free-sound-0306.wav
2 ../data/musan/noise/free-sound/noise-free-sound-0115.wav
3 ../data/musan/noise/free-sound/noise-free-sound-0566.wav
4 ../data/musan/noise/free-sound/noise-free-sound-0152.wav
5 ../data/musan/noise/free-sound/noise-free-sound-0527.wav
6 ../data/musan/noise/free-sound/noise-free-sound-0302.wav
7 ../data/musan/noise/free-sound/noise-free-sound-0461.wav
8 ../data/musan/noise/free-sound/noise-free-sound-0141.wav
9 ../data/musan/noise/free-sound/noise-free-sound-0565.wav
10 ../data/musan/noise/free-sound/noise-free-sound-0744.wav
11 ../data/musan/noise/free-sound/noise-free-sound-0336.wav
12 ../data/musan/noise/free-sound/noise-free-sound-0038.wav
13 ../data/musan/noise/free-sound/noise-free-sound-0727.wav
14 ../data/musan/noise/free-sound/noise-free-sound-0445.wav
15 ../data/musan/noise/free-sound/noise-free-sound-0762.wav
16 ../data/musan/noise/free-sound/noise-free-sound

201 ../data/musan/noise/free-sound/noise-free-sound-0059.wav
202 ../data/musan/noise/free-sound/noise-free-sound-0290.wav
203 ../data/musan/noise/free-sound/noise-free-sound-0396.wav
204 ../data/musan/noise/free-sound/noise-free-sound-0760.wav
205 ../data/musan/noise/free-sound/noise-free-sound-0528.wav
206 ../data/musan/noise/free-sound/noise-free-sound-0688.wav
207 ../data/musan/noise/free-sound/noise-free-sound-0021.wav
208 ../data/musan/noise/free-sound/noise-free-sound-0784.wav
209 ../data/musan/noise/free-sound/noise-free-sound-0755.wav
210 ../data/musan/noise/free-sound/noise-free-sound-0049.wav
211 ../data/musan/noise/free-sound/noise-free-sound-0795.wav
212 ../data/musan/noise/free-sound/noise-free-sound-0635.wav
213 ../data/musan/noise/free-sound/noise-free-sound-0430.wav
214 ../data/musan/noise/free-sound/noise-free-sound-0358.wav
215 ../data/musan/noise/free-sound/noise-free-sound-0477.wav
216 ../data/musan/noise/free-sound/noise-free-sound-0563.wav
217 ../data/musan/noise/

379 ../data/musan/noise/free-sound/noise-free-sound-0524.wav
380 ../data/musan/noise/free-sound/noise-free-sound-0791.wav
381 ../data/musan/noise/free-sound/noise-free-sound-0454.wav
382 ../data/musan/noise/free-sound/noise-free-sound-0397.wav
383 ../data/musan/noise/free-sound/noise-free-sound-0226.wav
384 ../data/musan/noise/free-sound/noise-free-sound-0186.wav
385 ../data/musan/noise/free-sound/noise-free-sound-0096.wav
386 ../data/musan/noise/free-sound/noise-free-sound-0765.wav
387 ../data/musan/noise/free-sound/noise-free-sound-0008.wav
388 ../data/musan/noise/free-sound/noise-free-sound-0801.wav
389 ../data/musan/noise/free-sound/noise-free-sound-0207.wav
390 ../data/musan/noise/free-sound/noise-free-sound-0790.wav
391 ../data/musan/noise/free-sound/noise-free-sound-0837.wav
392 ../data/musan/noise/free-sound/noise-free-sound-0706.wav
393 ../data/musan/noise/free-sound/noise-free-sound-0554.wav
394 ../data/musan/noise/free-sound/noise-free-sound-0112.wav
395 ../data/musan/noise/

607 ../data/musan/noise/free-sound/noise-free-sound-0225.wav
608 ../data/musan/noise/free-sound/noise-free-sound-0285.wav
609 ../data/musan/noise/free-sound/noise-free-sound-0628.wav
610 ../data/musan/noise/free-sound/noise-free-sound-0234.wav
611 ../data/musan/noise/free-sound/noise-free-sound-0067.wav
612 ../data/musan/noise/free-sound/noise-free-sound-0555.wav
613 ../data/musan/noise/free-sound/noise-free-sound-0517.wav
614 ../data/musan/noise/free-sound/noise-free-sound-0132.wav
615 ../data/musan/noise/free-sound/noise-free-sound-0264.wav
616 ../data/musan/noise/free-sound/noise-free-sound-0401.wav
617 ../data/musan/noise/free-sound/noise-free-sound-0792.wav
618 ../data/musan/noise/free-sound/noise-free-sound-0425.wav
619 ../data/musan/noise/free-sound/noise-free-sound-0224.wav
620 ../data/musan/noise/free-sound/noise-free-sound-0820.wav
621 ../data/musan/noise/free-sound/noise-free-sound-0009.wav
622 ../data/musan/noise/free-sound/noise-free-sound-0377.wav
623 ../data/musan/noise/

803 ../data/musan/noise/free-sound/noise-free-sound-0560.wav
804 ../data/musan/noise/free-sound/noise-free-sound-0595.wav
805 ../data/musan/noise/free-sound/noise-free-sound-0235.wav
806 ../data/musan/noise/free-sound/noise-free-sound-0039.wav
807 ../data/musan/noise/free-sound/noise-free-sound-0637.wav
808 ../data/musan/noise/free-sound/noise-free-sound-0490.wav
809 ../data/musan/noise/free-sound/noise-free-sound-0202.wav
810 ../data/musan/noise/free-sound/noise-free-sound-0449.wav
811 ../data/musan/noise/free-sound/noise-free-sound-0228.wav
812 ../data/musan/noise/free-sound/noise-free-sound-0381.wav
813 ../data/musan/noise/free-sound/noise-free-sound-0024.wav
814 ../data/musan/noise/free-sound/noise-free-sound-0180.wav
815 ../data/musan/noise/free-sound/noise-free-sound-0548.wav
816 ../data/musan/noise/free-sound/noise-free-sound-0095.wav
817 ../data/musan/noise/free-sound/noise-free-sound-0638.wav
818 ../data/musan/noise/free-sound/noise-free-sound-0027.wav
819 ../data/musan/noise/

...

1423 ../data/musan/music/rfm/music-rfm-0080.wav
1424 ../data/musan/music/rfm/music-rfm-0005.wav
1425 ../data/musan/music/rfm/music-rfm-0045.wav
1426 ../data/musan/music/rfm/music-rfm-0000.wav
1427 ../data/musan/music/rfm/music-rfm-0114.wav
1428 ../data/musan/music/rfm/music-rfm-0136.wav
1429 ../data/musan/music/rfm/music-rfm-0118.wav
1430 ../data/musan/music/rfm/music-rfm-0124.wav
1431 ../data/musan/music/rfm/music-rfm-0018.wav
1432 ../data/musan/music/rfm/music-rfm-0048.wav
1433 ../data/musan/music/rfm/music-rfm-0108.wav
1434 ../data/musan/music/rfm/music-rfm-0028.wav
1435 ../data/musan/music/rfm/music-rfm-0055.wav
1436 ../data/musan/music/rfm/music-rfm-0015.wav
1437 ../data/musan/music/rfm/music-rfm-0010.wav
1438 ../data/musan/music/rfm/music-rfm-0023.wav
1439 ../data/musan/music/rfm/music-rfm-0144.wav
1440 ../data/musan/music/rfm/music-rfm-0039.wav
1441 ../data/musan/music/rfm/music-rfm-0063.wav
1442 ../data/musan/music/rfm/music-rfm-0047.wav
1443 ../data/musan/music/rfm/music-rfm-0

1579 ../data/musan/music/fma-western-art/music-fma-wa-0029.wav
1580 ../data/musan/music/fma-western-art/music-fma-wa-0032.wav
1581 ../data/musan/music/fma-western-art/music-fma-wa-0005.wav
1582 ../data/musan/music/fma-western-art/music-fma-wa-0002.wav
1583 ../data/musan/music/fma-western-art/music-fma-wa-0033.wav
1584 ../data/musan/music/fma-western-art/music-fma-wa-0082.wav
1585 ../data/musan/music/fma-western-art/music-fma-wa-0003.wav
1586 ../data/musan/music/fma-western-art/music-fma-wa-0069.wav
1587 ../data/musan/music/fma-western-art/music-fma-wa-0070.wav
1588 ../data/musan/music/fma-western-art/music-fma-wa-0085.wav
1589 ../data/musan/music/fma-western-art/music-fma-wa-0010.wav
1590 ../data/musan/speech/librivox/speech-librivox-0134.wav
1591 ../data/musan/speech/librivox/speech-librivox-0005.wav
1592 ../data/musan/speech/librivox/speech-librivox-0013.wav
1593 ../data/musan/speech/librivox/speech-librivox-0171.wav
1594 ../data/musan/speech/librivox/speech-librivox-0117.wav
1595 ..

1715 ../data/musan/speech/librivox/speech-librivox-0033.wav
1716 ../data/musan/speech/librivox/speech-librivox-0062.wav
1717 ../data/musan/speech/librivox/speech-librivox-0054.wav
1718 ../data/musan/speech/librivox/speech-librivox-0010.wav
1719 ../data/musan/speech/librivox/speech-librivox-0045.wav
1720 ../data/musan/speech/librivox/speech-librivox-0124.wav
1721 ../data/musan/speech/librivox/speech-librivox-0148.wav
1722 ../data/musan/speech/librivox/speech-librivox-0000.wav
1723 ../data/musan/speech/librivox/speech-librivox-0028.wav
1724 ../data/musan/speech/librivox/speech-librivox-0001.wav
1725 ../data/musan/speech/librivox/speech-librivox-0096.wav
1726 ../data/musan/speech/librivox/speech-librivox-0023.wav
1727 ../data/musan/speech/librivox/speech-librivox-0099.wav
1728 ../data/musan/speech/librivox/speech-librivox-0043.wav
1729 ../data/musan/speech/librivox/speech-librivox-0006.wav
1730 ../data/musan/speech/librivox/speech-librivox-0167.wav
1731 ../data/musan/speech/librivox/speec

1860 ../data/musan/speech/us-gov/speech-us-gov-0205.wav
1861 ../data/musan/speech/us-gov/speech-us-gov-0187.wav
1862 ../data/musan/speech/us-gov/speech-us-gov-0249.wav
1863 ../data/musan/speech/us-gov/speech-us-gov-0193.wav
1864 ../data/musan/speech/us-gov/speech-us-gov-0129.wav
1865 ../data/musan/speech/us-gov/speech-us-gov-0055.wav
1866 ../data/musan/speech/us-gov/speech-us-gov-0112.wav
1867 ../data/musan/speech/us-gov/speech-us-gov-0247.wav
1868 ../data/musan/speech/us-gov/speech-us-gov-0242.wav
1869 ../data/musan/speech/us-gov/speech-us-gov-0094.wav
1870 ../data/musan/speech/us-gov/speech-us-gov-0090.wav
1871 ../data/musan/speech/us-gov/speech-us-gov-0211.wav
1872 ../data/musan/speech/us-gov/speech-us-gov-0217.wav
1873 ../data/musan/speech/us-gov/speech-us-gov-0121.wav
1874 ../data/musan/speech/us-gov/speech-us-gov-0125.wav
1875 ../data/musan/speech/us-gov/speech-us-gov-0140.wav
1876 ../data/musan/speech/us-gov/speech-us-gov-0123.wav
1877 ../data/musan/speech/us-gov/speech-us-gov-0

2008 ../data/musan/speech/us-gov/speech-us-gov-0173.wav
2009 ../data/musan/speech/us-gov/speech-us-gov-0038.wav
2010 ../data/musan/speech/us-gov/speech-us-gov-0137.wav
2011 ../data/musan/speech/us-gov/speech-us-gov-0114.wav
2012 ../data/musan/speech/us-gov/speech-us-gov-0229.wav
2013 ../data/musan/speech/us-gov/speech-us-gov-0072.wav
2014 ../data/musan/speech/us-gov/speech-us-gov-0023.wav
2015 ../data/musan/speech/us-gov/speech-us-gov-0222.wav


**2. Обучение параметров блока построения дикторских моделей без учёта процедуры аугментации данных**

Построение современных дикторских моделей, как правило, выполняется с использованием нейросетевых архитектур. В последние годы наилучшие результаты демонстрируют [трансформерные модели](https://arxiv.org/pdf/1706.03762), существенно превосходящие по качеству более ранние свёрточные нейросетевые архитектуры. В рамках настоящего пункта предлагается выполнить адаптацию нейросетевой архитектуры [Whisper tiny](https://arxiv.org/pdf/2212.04356) под решение задачи генерации дикторских моделей (дикторских эмбеддингов). *Дикторский эмбеддинг* – это высокоуровневый вектор-признаков, состоящий, например, из 128, 256 и т.п. значений, содержащий особенности голоса конкретного человека. При решении задачи распознавания диктора можно выделить эталонные и тестовые дикторские эмбеддинги. *Эталонные эмбеддинги* формируются на этапе регистрации дикторской модели определённого человека и находятся в некотором хранилище данных. *Тестовые эмбеддинги* формируются на этапе непосредственного использования системы голосовой биометрии на практике, когда некоторый пользователь пытается получить доступ к соответствующим ресурсам. Система голосовой биометрии сравнивает по определённой метрике эталонные и тестовые эмбеддинги, формируя оценку сравнения, которая, после её обработки блоком принятия решения, позволяет сделать вывод о том, эмбеддинги одинаковых или разных дикторов сравниваются между собой.

Необходимо отметить, что построение дикторских моделей, как правило, требует наличия *акустических признаков*, вычисленных для звукозаписей тренировочной, валидационной и тестовой баз данных. В качестве примера подобных признаков в рамках настоящей лабораторной работы воспользуемся *логарифмами энергий на выходе мел-банка фильтров*. Важно отметить, что оригинальная модель Whisper tiny обучалась на 80-мерных признаках, причём входной сигнал перед обработкой путём дополнения с краю приводится к фиксированной длительности равной 30 секундам. В рамках настоящей лабораторной работы дополнение сигнала до 30 секунд не выполняется. Дополнительно, перед вычислением акустических признаков, осуществляется применение [нормализации экземпляра](https://docs.pytorch.org/docs/stable/generated/torch.nn.InstanceNorm1d.html) (instance normalization), которая выравнивает амплитуды всех сигналов в тренировочной выборке, стабилизируя процедуру сходимости нейросетевой модели, выполняющей построение дикторских моделей, на этапе обучения. Подобная нормализация отсутствует в процедуре обучения оригинальных моделей семейства Whisper.

После того, как акустические признаки подготовлены, они могут быть переданы на блок построения дикторских моделей. Как правило, устройство современных дикторских моделей соответствует структуре [x-векторных архитектур](https://www.danielpovey.com/files/2018_icassp_xvectors.pdf). Эти архитектуры состоят из четырёх ключевых элементов: 

1. **Фреймовый уровень.** Предназначен для формирования локальных представлений голоса конкретного человека. На этом уровне как раз и могут быть использованы нейросетевые архитектуры на базе трансормерных моделей. При организации фреймового уровня в рамках настоящей лабораторной работы предлагается использовать предобученный кодировщик трансформерной нейросетевой архитектуры Whisper tiny, а также два [нейросетевых блока с временной задержкой](https://www.cs.toronto.edu/~hinton/absps/waibelTDNN.pdf) (TDNN, time-delay neural betwork), которые выполняют согласование кодировщика Whisper tiny с оставшейся частью нейронной сети. Использование мощных, предварительно обученных на большом объёме размеченных или неразмеченных данных трансформерных моделей, таких как [wav2vec 2.0](https://arxiv.org/pdf/2006.11477), [HuBERT](https://arxiv.org/pdf/2106.07447), [Whisper](https://arxiv.org/pdf/2212.04356) и т.п., значительно упрощает решение множества разнообразных *целевых задач* (downstream tasks), связанных с обработкой аудиосигналов, ускоряя процедуру сходимости на этапе обучения и улучшая качество работы моделей в процессе тестирования относительно моделей, обученных из случайной начальной инициализации.

2. **Уровень статистического пулинга** позволяет сформировать промежуточный вектор-признаков, фиксированной длины, которая является одинаковой для звукозаписи любой длительности. В ходе работы блока статистического пулинга происходит удаление временной размерности, присутствующей в картах-признаков. Это достигается путём выполнения процедуры усреднения карт-признаков вдоль оси времени. Выходом уровня статистического пулинга являются вектор среднего и вектор среднеквадратического отклонения, вычисленные на основе карт-признаков. Эти вектора конкатенируются и передаются для дальнейшей обработки на сегментом уровне.

3. **Сегментный уровень.** Предназначен для трансформации промежуточного вектора, как правило, высокой размерности, в компактный вектор-признаков, представляющий собой дикторский эмбеддинг. Необходимо отметить, что на сегментном уровне расположены один или несколько полносвязных нейросетевых слоёв, а обработка данных выполняется по отношению ко всей звукозаписи, а не только к некоторому её локальному контексту, как на фреймовом уровне.

4. **Уровень выходного слоя.** Представляет полносвязный слой с softmax-функциями активации. Количество активаций равно числу дикторов в тренирочной выборке. На вход выходноя слоя подаётся дикторский эмбеддинг, а на выходе – формируется набор апостериорных вероятностей, определяющих принадлежность эмбеддинга к одному из дикторских классов в тренировочной выборке. Необходимо отметить, что, как правило, в современных нейросетевых системах построения дикторских моделей выходной слой используется только на этапе обучения параметров и на этапе тестирования не применяется. На этапе тестирования используются только три первых уровня архитектуры.

Обучение модели генерации дикторских эмбеддингов выполняется путём решения задачи *классификации* или, выражаясь терминами из области биометрии, *идентификации на закрытом множестве* (количество дикторских меток является строго фиксированным). В качестве используемой стоимостной функции выступает *категориальная кросс-энтропия*. Обучение выполняется с помощью мини-батчей, содержащих короткие фрагменты карт акустических признаков (длительностью несколько секунд) различных дикторов из тренировочной базы данных. Обучение на коротких фрагментах позволяет избежать сильного переобучения нейросетевой модели. При выполнении процедуры обучения требуется подобрать набор гиперпараметров, выбрать обучения и метод численной оптимизации. Обучение нейросетевых моделей при наличие претрейнов добавляет определённую специфику в процедуру обучения: использование более низкого значения параметра сходимости в случае, если вся нейросетевая модель *«разморожена»* (обучаются все веса модели на этапе тренировки), *«заморозка»* предобученной части модели (обучается часть весов модели на этапе тренировки) для ускорения процедуры сходимости и быстрой проверки гипотез и т.п.

Для успешного выполнения настоящего пункта необходимо сделать следующее:

1. Сгенерировать списки тренировочных и валидационных данных на основе идентификационного протокола базы VoxCeleb1, содержащегося в файле **../data/voxceleb1_test/iden_split.txt**. При генерации списков требуется исключить из них звукозаписи дикторов, которые входят в базу [VoxCeleb1 test set](https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip). Это позволит выполнить тестирования обученных блоков генерации дикторских моделей на протоколе [VoxCeleb1-O cleaned](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test2.txt), который составлен по отношению к данным из VoxCeleb1 test set и позволяет проверить качество работы биометрической системы в рамках открытой задачи верификации диктора.

2. Инициализировать обучаемый экстрактор дикторских эмбеддингов и вгрузить в его фреймовый уровень веса предобученного кодировщика модели Whisper tiny. Необходимо отметить, что Whisper tiny не является единственной моделью семейства [Whisper](https://github.com/openai/whisper/tree/main). Так же в общедоступных источниках могут быть найдены веса предобученных моделей Whisper base/small/medium/large. Однако, вычислительная стоимость этих моделей в сравнении с Whisper tiny является выше при более высокой точности. Для достижения более высокого качества экстрактора дикторских эмбеддингов предлагается разморозить слои кодировщика Whisper tiny на этапе обучения. 

3. Инициализировать загрузчики тренировочной и валидационной выборок, используемые в рамках решения задачи идентификации на закрытом множестве, а также тестовой выборки, для оценки ошибки системы распознавания в рамках решения задачи верификации.

4. Инициализировать оптимизатор и планировщик для выполнения процедуры обучения.

5. Описать процедуру валидации блока построения дикторских моделей.

6. Описать процедуру обучения и запустить её, контролируя значения стоимостной функции и доли правильных ответов на тренировочном и валидационном множествах, а также величину равновероятной ошибки при решении задачи верификации на тестовом множестве.

In [9]:
# Select hyperparameters

nOut              = 512                                  # embedding size

# Loss function for angular losses
margin            = 0.35                                 # margin parameter
scale             = 32.0                                 # scale parameter

# Train dataloader (close-set identification)
max_frames_train  = 400                                  # number of frame to train
train_path        = '../data/voxceleb1_dev/wav'          # path to train wav files
batch_size_train  = 128                                  # batch size to train
pin_memory        = False                                # pin memory
num_workers_train = 5                                    # number of workers to train
shuffle           = True                                 # shuffling of training examples

# Validation dataloader (close-set identification)
max_frames_val    = 400                                  # number of frame to validate
val_path          = '../data/voxceleb1_dev/wav'          # path to val wav files
batch_size_val    = 128                                  # batch size to validate
num_workers_val   = 5                                    # number of workers to validate

# Test dataloader (open-set verification)
max_frames_test   = 400                                  # number of frame to test
test_path         = '../data/voxceleb1_test/wav/'        # path to val wav files
batch_size_test   = 1                                    # batch size to test
num_workers_test  = 5                                    # number of workers to test
num_eval          = 5                                    # number of enroll/test chunks for scoring

# Optimizer
lr                = 0.0001                               # learning rate value # 0.0001
weight_decay      = 0                                    # weight decay value

# Scheduler
val_interval      = 5                                    # frequency of validation step
max_epoch         = 40                                   # number of epoches

# Augmentation
musan_path        = '../data/musan_split'                # path to splitted SLR17 dataset
rir_path          = '../data/RIRS_NOISES/simulated_rirs' # path to SLR28 dataset

In [10]:
# Generate train/validation data lists for close-set speaker identification
train_list = []
val_list   = []

with open('../data/voxceleb1_test/iden_split.txt', 'r') as f:
    lines = f.readlines()
    
black_list = os.listdir('../data/voxceleb1_test/wav')   # exclude speaker IDs from VoxCeleb1 test set
num_train_spk = []                                      # number of train speakers

for line in lines:
    line   = line.strip().split(' ')
    spk_id = line[1].split('/')[0]
    
    if not (spk_id in black_list):
        num_train_spk.append(spk_id)
        
    else:
        continue
    
    # Train list
    if (line[0] == '1'):
        train_list.append(' '.join([spk_id, line[1]]))
    
    # Validation list
    elif (line[0] == '2'):
        val_list.append(' '.join([spk_id, line[1]]))
        
num_train_spk = len(set(num_train_spk))

In [11]:
# Initialize train dataloader (with augmentation)
train_dataset = train_dataset_loader(train_list=train_list, 
                                     max_frames=max_frames_train, 
                                     train_path=train_path, 
                                     augment=True, 
                                     musan_path=musan_path, 
                                     rir_path=rir_path)

train_loader  = DataLoader(train_dataset, 
                           batch_size=batch_size_train, 
                           pin_memory=pin_memory, 
                           num_workers=num_workers_train, 
                           shuffle=shuffle)

# Initialize validation dataloader
val_dataset = test_dataset_loader(test_list=val_list, max_frames=max_frames_val, test_path=val_path)
val_loader  = DataLoader(val_dataset, batch_size=batch_size_val, num_workers=num_workers_val)

In [12]:
# Load test protocol
with open('../data/voxceleb1_test/veri_test2.txt', 'r') as f:
    verif_protocol_lines = f.readlines()
    
# Get a list of unique file names
files = list(itertools.chain(*[x.strip().split()[-2:] for x in verif_protocol_lines]))
setfiles = list(set(files))
setfiles.sort()

# Initialize test dataloader
test_dataset = test_dataset_loader_verification(setfiles, test_path=test_path, eval_frames=max_frames_test, num_eval=num_eval)
test_loader  = DataLoader(test_dataset, 
                          batch_size=batch_size_test, 
                          shuffle=False, 
                          num_workers=num_workers_test, 
                          drop_last=False, 
                          sampler=None)

In [13]:
# Initialize model
preprocessor        = ExtractWhisperFbanks80(pad_data=False)
feat_extractor      = WhisperEncoder.build_model(model_name="tiny", load_weight=True, n_layer=-1) # Whisper tiny download to 
                                                                                                  # ~/.cache/whisper/tiny.pt
frame_level_block   = TDNNSimple(n_layers=1, in_ch=384, out_ch=1024)
pooling_block       = StatPoolLayer.by_string_mode(mode="MV")
segment_level_block = MaxoutSegmentLevel(input_dim=2048, output_dim=512, enable_batch_norm=True)

whisper_sr_model    = SSLDownstream(preproc=preprocessor,
                                    feat_extractor=feat_extractor,
                                    frame_level_block=frame_level_block,
                                    pooling_block=pooling_block,
                                    segment_level_block=segment_level_block,
                                    freeze_feats=False)
trainfunc           = AAMSoftmaxLoss(nOut=nOut, nClasses=num_train_spk, margin=margin, scale=scale)
main_model          = MainModel(whisper_sr_model, trainfunc).cuda()

100%|█████████████████████████████████████| 72.1M/72.1M [00:02<00:00, 28.3MiB/s]


Initialised AAM softmax margin 0.350 scale 32.000.


In [14]:
# Initialize optimizer and scheduler
optimizer = SGDOptimizer(main_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = OneCycleLRScheduler(optimizer, 
                                pct_start=0.05, 
                                cycle_momentum=False, 
                                max_lr=lr, 
                                div_factor=10, 
                                final_div_factor=10000, 
                                total_steps=max_epoch*len(train_loader))

Initialised SGD optimizer.
Initialised OneCycle LR scheduler.


In [15]:
start_epoch = 0
checkpoint_flag = False

if checkpoint_flag:
    start_epoch = loadParameters(main_model, optimizer, scheduler, path='../data/lab6_models/lab6_model_0004.pth')
    start_epoch = start_epoch + 1

# Train model
for num_epoch in range(start_epoch, max_epoch):
    train_loss, train_top1 = train_network(train_loader, main_model, optimizer, scheduler, num_epoch, verbose=True)
    
    print("Epoch {:1.0f}, Loss (train set) {:f}, Accuracy (train set) {:2.3f}%".format(num_epoch, train_loss, train_top1))

    if (num_epoch + 1)%val_interval == 0:
        _, val_top1 = test_network(val_loader, main_model)                               # compute accuracy for validation set
        
        print("Epoch {:1.0f}, Accuracy (validation set) {:2.3f}%".format(num_epoch, val_top1))
        
        feats = extract_features(main_model, test_loader)                                # compute enroll/test speaker embeddings
        all_scores, all_labels, all_trials = compute_scores(feats, verif_protocol_lines) # compute scores between enroll and test speaker models
        tar_scores, imp_scores = tar_imp_hists(all_scores, all_labels)                   # devide the scores into target and impostor scores
        EER, _ = get_eer(tar_scores, imp_scores)                                         # compute EER, %
        
        print("Epoch {:1.0f}, EER (test set) {:2.3f}%".format(num_epoch, EER))
        
        saveParameters(main_model, optimizer, scheduler, num_epoch, path='../data/lab6_models')

Epoch 0, Batch 1, LR 0.000010 Loss 19.094431, Accuracy 0.000%
Epoch 0, Batch 2, LR 0.000010 Loss 19.085585, Accuracy 0.000%
Epoch 0, Batch 3, LR 0.000010 Loss 19.000217, Accuracy 0.000%
Epoch 0, Batch 4, LR 0.000010 Loss 18.988174, Accuracy 0.000%
Epoch 0, Batch 5, LR 0.000010 Loss 18.989373, Accuracy 0.000%
Epoch 0, Batch 6, LR 0.000010 Loss 19.016659, Accuracy 0.000%
Epoch 0, Batch 7, LR 0.000010 Loss 18.974042, Accuracy 0.112%
Epoch 0, Batch 8, LR 0.000010 Loss 18.988524, Accuracy 0.098%
Epoch 0, Batch 9, LR 0.000010 Loss 18.991696, Accuracy 0.087%
Epoch 0, Batch 10, LR 0.000010 Loss 18.981946, Accuracy 0.078%
Epoch 0, Batch 11, LR 0.000010 Loss 18.988835, Accuracy 0.071%
Epoch 0, Batch 12, LR 0.000010 Loss 18.987102, Accuracy 0.065%
Epoch 0, Batch 13, LR 0.000010 Loss 18.988402, Accuracy 0.060%
Epoch 0, Batch 14, LR 0.000010 Loss 18.991361, Accuracy 0.112%
Epoch 0, Batch 15, LR 0.000010 Loss 18.993636, Accuracy 0.104%
Epoch 0, Batch 16, LR 0.000010 Loss 19.006969, Accuracy 0.098%
E

Epoch 0, Batch 131, LR 0.000011 Loss 18.991395, Accuracy 0.101%
Epoch 0, Batch 132, LR 0.000011 Loss 18.992251, Accuracy 0.101%
Epoch 0, Batch 133, LR 0.000011 Loss 18.991399, Accuracy 0.100%
Epoch 0, Batch 134, LR 0.000011 Loss 18.990885, Accuracy 0.099%
Epoch 0, Batch 135, LR 0.000011 Loss 18.989703, Accuracy 0.104%
Epoch 0, Batch 136, LR 0.000011 Loss 18.988801, Accuracy 0.103%
Epoch 0, Batch 137, LR 0.000011 Loss 18.989222, Accuracy 0.103%
Epoch 0, Batch 138, LR 0.000011 Loss 18.989387, Accuracy 0.102%
Epoch 0, Batch 139, LR 0.000011 Loss 18.989236, Accuracy 0.101%
Epoch 0, Batch 140, LR 0.000011 Loss 18.990814, Accuracy 0.100%
Epoch 0, Batch 141, LR 0.000011 Loss 18.991747, Accuracy 0.100%
Epoch 0, Batch 142, LR 0.000011 Loss 18.991525, Accuracy 0.099%
Epoch 0, Batch 143, LR 0.000011 Loss 18.990531, Accuracy 0.098%
Epoch 0, Batch 144, LR 0.000011 Loss 18.990469, Accuracy 0.098%
Epoch 0, Batch 145, LR 0.000011 Loss 18.989457, Accuracy 0.097%
Epoch 0, Batch 146, LR 0.000011 Loss 18.

Epoch 0, Batch 260, LR 0.000013 Loss 18.932750, Accuracy 0.120%
Epoch 0, Batch 261, LR 0.000013 Loss 18.933029, Accuracy 0.120%
Epoch 0, Batch 262, LR 0.000013 Loss 18.932906, Accuracy 0.119%
Epoch 0, Batch 263, LR 0.000013 Loss 18.931665, Accuracy 0.119%
Epoch 0, Batch 264, LR 0.000013 Loss 18.931430, Accuracy 0.118%
Epoch 0, Batch 265, LR 0.000013 Loss 18.931025, Accuracy 0.118%
Epoch 0, Batch 266, LR 0.000014 Loss 18.930666, Accuracy 0.117%
Epoch 0, Batch 267, LR 0.000014 Loss 18.929953, Accuracy 0.120%
Epoch 0, Batch 268, LR 0.000014 Loss 18.928973, Accuracy 0.120%
Epoch 0, Batch 269, LR 0.000014 Loss 18.928628, Accuracy 0.119%
Epoch 0, Batch 270, LR 0.000014 Loss 18.928197, Accuracy 0.119%
Epoch 0, Batch 271, LR 0.000014 Loss 18.927633, Accuracy 0.118%
Epoch 0, Batch 272, LR 0.000014 Loss 18.927108, Accuracy 0.118%
Epoch 0, Batch 273, LR 0.000014 Loss 18.926572, Accuracy 0.117%
Epoch 0, Batch 274, LR 0.000014 Loss 18.925583, Accuracy 0.117%
Epoch 0, Batch 275, LR 0.000014 Loss 18.

Epoch 0, Batch 389, LR 0.000017 Loss 18.846912, Accuracy 0.177%
Epoch 0, Batch 390, LR 0.000017 Loss 18.846428, Accuracy 0.176%
Epoch 0, Batch 391, LR 0.000017 Loss 18.845804, Accuracy 0.176%
Epoch 0, Batch 392, LR 0.000018 Loss 18.845204, Accuracy 0.175%
Epoch 0, Batch 393, LR 0.000018 Loss 18.844888, Accuracy 0.175%
Epoch 0, Batch 394, LR 0.000018 Loss 18.844568, Accuracy 0.174%
Epoch 0, Batch 395, LR 0.000018 Loss 18.843977, Accuracy 0.174%
Epoch 0, Batch 396, LR 0.000018 Loss 18.843172, Accuracy 0.180%
Epoch 0, Batch 397, LR 0.000018 Loss 18.842533, Accuracy 0.179%
Epoch 0, Batch 398, LR 0.000018 Loss 18.841378, Accuracy 0.181%
Epoch 0, Batch 399, LR 0.000018 Loss 18.840266, Accuracy 0.182%
Epoch 0, Batch 400, LR 0.000018 Loss 18.839765, Accuracy 0.184%
Epoch 0, Batch 401, LR 0.000018 Loss 18.838706, Accuracy 0.185%
Epoch 0, Batch 402, LR 0.000018 Loss 18.838511, Accuracy 0.185%
Epoch 0, Batch 403, LR 0.000018 Loss 18.838162, Accuracy 0.188%
Epoch 0, Batch 404, LR 0.000018 Loss 18.

Epoch 0, Batch 518, LR 0.000023 Loss 18.740424, Accuracy 0.350%
Epoch 0, Batch 519, LR 0.000023 Loss 18.739587, Accuracy 0.351%
Epoch 0, Batch 520, LR 0.000023 Loss 18.738864, Accuracy 0.352%
Epoch 0, Batch 521, LR 0.000023 Loss 18.737862, Accuracy 0.352%
Epoch 0, Batch 522, LR 0.000023 Loss 18.737096, Accuracy 0.353%
Epoch 0, Batch 523, LR 0.000023 Loss 18.736145, Accuracy 0.356%
Epoch 0, Batch 524, LR 0.000023 Loss 18.735064, Accuracy 0.355%
Epoch 0, Batch 525, LR 0.000023 Loss 18.734261, Accuracy 0.356%
Epoch 0, Batch 526, LR 0.000023 Loss 18.733333, Accuracy 0.356%
Epoch 0, Batch 527, LR 0.000023 Loss 18.732339, Accuracy 0.362%
Epoch 0, Batch 528, LR 0.000023 Loss 18.731287, Accuracy 0.368%
Epoch 0, Batch 529, LR 0.000023 Loss 18.730543, Accuracy 0.369%
Epoch 0, Batch 530, LR 0.000023 Loss 18.729657, Accuracy 0.369%
Epoch 0, Batch 531, LR 0.000024 Loss 18.728807, Accuracy 0.372%
Epoch 0, Batch 532, LR 0.000024 Loss 18.727717, Accuracy 0.377%
Epoch 0, Batch 533, LR 0.000024 Loss 18.

...

Epoch 39, Batch 450, LR 0.000000 Loss 4.419842, Accuracy 89.847%
Epoch 39, Batch 451, LR 0.000000 Loss 4.419098, Accuracy 89.852%
Epoch 39, Batch 452, LR 0.000000 Loss 4.417892, Accuracy 89.858%
Epoch 39, Batch 453, LR 0.000000 Loss 4.418728, Accuracy 89.845%
Epoch 39, Batch 454, LR 0.000000 Loss 4.419250, Accuracy 89.842%
Epoch 39, Batch 455, LR 0.000000 Loss 4.418429, Accuracy 89.849%
Epoch 39, Batch 456, LR 0.000000 Loss 4.417258, Accuracy 89.854%
Epoch 39, Batch 457, LR 0.000000 Loss 4.417798, Accuracy 89.847%
Epoch 39, Batch 458, LR 0.000000 Loss 4.418270, Accuracy 89.849%
Epoch 39, Batch 459, LR 0.000000 Loss 4.417710, Accuracy 89.852%
Epoch 39, Batch 460, LR 0.000000 Loss 4.418782, Accuracy 89.839%
Epoch 39, Batch 461, LR 0.000000 Loss 4.418801, Accuracy 89.834%
Epoch 39, Batch 462, LR 0.000000 Loss 4.418835, Accuracy 89.829%
Epoch 39, Batch 463, LR 0.000000 Loss 4.420508, Accuracy 89.818%
Epoch 39, Batch 464, LR 0.000000 Loss 4.420483, Accuracy 89.817%
Epoch 39, Batch 465, LR 0

Epoch 39, Batch 577, LR 0.000000 Loss 4.416210, Accuracy 89.775%
Epoch 39, Batch 578, LR 0.000000 Loss 4.415891, Accuracy 89.775%
Epoch 39, Batch 579, LR 0.000000 Loss 4.416327, Accuracy 89.767%
Epoch 39, Batch 580, LR 0.000000 Loss 4.417270, Accuracy 89.760%
Epoch 39, Batch 581, LR 0.000000 Loss 4.417053, Accuracy 89.763%
Epoch 39, Batch 582, LR 0.000000 Loss 4.417246, Accuracy 89.763%
Epoch 39, Batch 583, LR 0.000000 Loss 4.417509, Accuracy 89.763%
Epoch 39, Batch 584, LR 0.000000 Loss 4.418205, Accuracy 89.769%
Epoch 39, Batch 585, LR 0.000000 Loss 4.417903, Accuracy 89.769%
Epoch 39, Batch 586, LR 0.000000 Loss 4.417781, Accuracy 89.769%
Epoch 39, Batch 587, LR 0.000000 Loss 4.418081, Accuracy 89.768%
Epoch 39, Batch 588, LR 0.000000 Loss 4.417426, Accuracy 89.771%
Epoch 39, Batch 589, LR 0.000000 Loss 4.417340, Accuracy 89.775%
Epoch 39, Batch 590, LR 0.000000 Loss 4.416454, Accuracy 89.779%
Epoch 39, Batch 591, LR 0.000000 Loss 4.415673, Accuracy 89.780%
Epoch 39, Batch 592, LR 0

Epoch 39, Batch 704, LR 0.000000 Loss 4.410673, Accuracy 89.777%
Epoch 39, Batch 705, LR 0.000000 Loss 4.410902, Accuracy 89.774%
Epoch 39, Batch 706, LR 0.000000 Loss 4.411125, Accuracy 89.773%
Epoch 39, Batch 707, LR 0.000000 Loss 4.410370, Accuracy 89.775%
Epoch 39, Batch 708, LR 0.000000 Loss 4.410235, Accuracy 89.772%
Epoch 39, Batch 709, LR 0.000000 Loss 4.410618, Accuracy 89.772%
Epoch 39, Batch 710, LR 0.000000 Loss 4.410115, Accuracy 89.773%
Epoch 39, Batch 711, LR 0.000000 Loss 4.410553, Accuracy 89.770%
Epoch 39, Batch 712, LR 0.000000 Loss 4.410520, Accuracy 89.775%
Epoch 39, Batch 713, LR 0.000000 Loss 4.410289, Accuracy 89.775%
Epoch 39, Batch 714, LR 0.000000 Loss 4.410108, Accuracy 89.774%
Epoch 39, Batch 715, LR 0.000000 Loss 4.409809, Accuracy 89.774%
Epoch 39, Batch 716, LR 0.000000 Loss 4.410156, Accuracy 89.773%
Epoch 39, Batch 717, LR 0.000000 Loss 4.410488, Accuracy 89.772%
Epoch 39, Batch 718, LR 0.000000 Loss 4.410993, Accuracy 89.773%
Epoch 39, Batch 719, LR 0

Epoch 39, Batch 831, LR 0.000000 Loss 4.414121, Accuracy 89.745%
Epoch 39, Batch 832, LR 0.000000 Loss 4.414298, Accuracy 89.743%
Epoch 39, Batch 833, LR 0.000000 Loss 4.413580, Accuracy 89.751%
Epoch 39, Batch 834, LR 0.000000 Loss 4.414109, Accuracy 89.749%
Epoch 39, Batch 835, LR 0.000000 Loss 4.413701, Accuracy 89.751%
Epoch 39, Batch 836, LR 0.000000 Loss 4.413613, Accuracy 89.753%
Epoch 39, Batch 837, LR 0.000000 Loss 4.414387, Accuracy 89.745%
Epoch 39, Batch 838, LR 0.000000 Loss 4.414995, Accuracy 89.742%
Epoch 39, Batch 839, LR 0.000000 Loss 4.415776, Accuracy 89.737%
Epoch 39, Batch 840, LR 0.000000 Loss 4.415708, Accuracy 89.736%
Epoch 39, Batch 841, LR 0.000000 Loss 4.415336, Accuracy 89.734%
Epoch 39, Batch 842, LR 0.000000 Loss 4.415527, Accuracy 89.735%
Epoch 39, Batch 843, LR 0.000000 Loss 4.414594, Accuracy 89.739%
Epoch 39, Batch 844, LR 0.000000 Loss 4.415088, Accuracy 89.739%
Epoch 39, Batch 845, LR 0.000000 Loss 4.415240, Accuracy 89.738%
Epoch 39, Batch 846, LR 0

Epoch 39, Batch 958, LR 0.000000 Loss 4.415315, Accuracy 89.708%
Epoch 39, Batch 959, LR 0.000000 Loss 4.415762, Accuracy 89.709%
Epoch 39, Batch 960, LR 0.000000 Loss 4.415128, Accuracy 89.713%
Epoch 39, Batch 961, LR 0.000000 Loss 4.415222, Accuracy 89.709%
Epoch 39, Batch 962, LR 0.000000 Loss 4.415371, Accuracy 89.708%
Epoch 39, Batch 963, LR 0.000000 Loss 4.414906, Accuracy 89.712%
Epoch 39, Batch 964, LR 0.000000 Loss 4.414703, Accuracy 89.712%
Epoch 39, Batch 965, LR 0.000000 Loss 4.414999, Accuracy 89.708%
Epoch 39, Batch 966, LR 0.000000 Loss 4.415051, Accuracy 89.707%
Epoch 39, Batch 967, LR 0.000000 Loss 4.414675, Accuracy 89.707%
Epoch 39, Batch 968, LR 0.000000 Loss 4.415120, Accuracy 89.703%
Epoch 39, Batch 969, LR 0.000000 Loss 4.415516, Accuracy 89.699%
Epoch 39, Batch 970, LR 0.000000 Loss 4.415427, Accuracy 89.700%
Epoch 39, Batch 971, LR 0.000000 Loss 4.415843, Accuracy 89.696%
Epoch 39, Batch 972, LR 0.000000 Loss 4.416312, Accuracy 89.694%
Epoch 39, Batch 973, LR 0

**3. Дополнительные задания по обучению дикторской модели на основе трансформерной архитектуры**

В рамках настоящего пункта предлагается выбрать и решить одно из дополнительных заданий на выбор:

1. Переведя параметр load_weight кодировщика Whisper tiny в режим False, попробуйте обучить трансформерную нейронную сеть без предварительно загруженных в неё весов кодировщика. Сравните полученное качество модели с тем, что было в пункте 2. При проведении исследования разрешается менять любые настройки процедуры обучения, в частности, параметр сходимости.

2. Переведя параметр freeze_feats кодировщика Whisper tiny в режим True, попробуйте обучить трансформерную нейронную сеть с «замороженным» кодировщиком. Сравните полученное качество модели с тем, что было в пункте 2. При проведении исследования разрешается менять любые настройки процедуры обучения, в частности, параметр сходимости.

3. В настройках процедуры обучения рассогласуйте параметр, отвечающий за максимальное число фреймов, используемых на этапе обучения, валидации и тестирования, например, выставив его равным 400 на этапе обучения и 1000 на этапах валидации и тестирования. Попробуйте обучить трансформерную нейронную сеть для выбранных настроек. Сравните полученное качество модели с тем, что было в пункте 2.

4. Попробуйте заменить кодировщика Whisper tiny на Whisper модель больших размеров base/small/medium/large и повторите процедуру обучения из пункта 2. Сравните полученное качество модели с тем, что было в пункте 2.

**4. Контрольные вопросы**

1. Что такое трансформерные нейросетевые модели и для какой задачи они были изначально предложены?

2. Назовите основные отличия трансформерных нейросетевых архитектур от свёрточных?

3. Как устроена архитектура Whisper и для решения каких задач она была изначально разработана?

4. Из каких основных нейросетевых слоёв состоит архитектура Whisper?

5. Какие разновидности архитектуры Whisper существуют и чем они отличаются друг от друга?

6. Что такое предобученная нейросетевая модель и для каких целей она используется?

7. В чём состоит основное отличие в политике изменения параметра сходимости на этапе обучения при наличии преобученной модели от ситуации, когда она отсутствует? 

8. Назовите примеры трансформерных нейросетевых претрейнов для обработки аудиосигналов и кратко опишите, как они были получены?

9. Как реализовать блок построения дикторских моделей на основе архитектуры Whisper?

**5. Список литературы**

1. Bai Z., Zhang X.-L., Chen J. Speaker recognition based on deep learning: an overview // 	arXiv:2012.00931 [eess.AS] ([ссылка](https://arxiv.org/pdf/2012.00931.pdf)).

2. Hansen J.H.L., Hasan T. Speaker recognition by machines and humans: a tutorial review // IEEE Signal Processing Magazine, 2015. V. 32. № 6. P. 74–99 ([ссылка](https://www.researchgate.net/publication/282940395_Speaker_Recognition_by_Machines_and_Humans_A_tutorial_review)).

3. Vaswani A. et al. Attention is all you need // arXiv:1706.03762 [cs.CL] ([ссылка](https://arxiv.org/pdf/1706.03762)).

4. Radford A. et al. Robust speech recognition via large-scale weak supervision // arXiv:2212.04356 [eess.AS] ([ссылка](https://arxiv.org/pdf/2212.04356)).